In [1]:
import pandas as pd
import numpy as np

In [49]:
# Read in dataset and remove uneccesary columns and stats
data = pd.read_csv("phs_2020/phs_2020_1.csv").drop(['start_time', 'tournament_title'], axis=1)

cols = ['esports_match_id', 'map_type', 'map_name', 'player_name', 'team_name', 'hero_name', 'stat_name', 'stat_amount']
data = data.reindex(columns = cols)
data = data[data["hero_name"] != "All Heroes"]
stats = ['Hero Damage Done', 'Assists', 'Barrier Damage Done', 'Deaths', 'Eliminations', 'Objective Kills', 'Objective Time', 'Ultimates Earned - Fractional', 'Final Blows', 'Damage Blocked', 'Healing Done']
data = data.loc[data['stat_name'].isin(stats)]
data['stat_amount'] = data['stat_amount'].round(decimals = 2)

# Tidy dataset to elongate stat_name column into rows and insert respective stat_amount values
data = data[['stat_name', 'stat_amount', 'player_name','esports_match_id','hero_name','map_name', 'team_name']].copy().reset_index()
data = data[['stat_name', 'stat_amount', 'player_name','esports_match_id','hero_name','map_name', 'team_name']]
data = data.pivot_table(values='stat_amount', index=[data['esports_match_id'],data['team_name'], data['player_name'], data['map_name'], data['hero_name']], columns='stat_name', aggfunc='sum')
data = data.fillna(0)
data = data.reset_index()
data.rename(columns = {'Ultimates Earned - Fractional':'Ultimates Earned'}, inplace = True)
data

stat_name,esports_match_id,team_name,player_name,map_name,hero_name,Assists,Barrier Damage Done,Damage Blocked,Deaths,Eliminations,Final Blows,Healing Done,Hero Damage Done,Objective Kills,Objective Time,Ultimates Earned
0,30991,Paris Eternal,BenBest,Eichenwalde,Reinhardt,10.0,5517.72,32655.81,9.0,19.0,9.0,0.00,8976.92,10.0,295.96,7.97
1,30991,Paris Eternal,BenBest,Havana,Reinhardt,6.0,3075.00,21137.18,11.0,10.0,4.0,0.00,6837.43,3.0,148.89,5.15
2,30991,Paris Eternal,BenBest,Horizon Lunar Colony,Reinhardt,13.0,3236.46,19394.67,10.0,21.0,8.0,0.00,8945.56,8.0,114.43,6.52
3,30991,Paris Eternal,BenBest,Lijiang Tower,Reinhardt,11.0,1875.00,14640.91,5.0,13.0,2.0,0.00,4798.16,9.0,65.05,3.71
4,30991,Paris Eternal,FDGod,Eichenwalde,Lúcio,14.0,5175.00,0.00,9.0,18.0,2.0,9229.73,6836.14,9.0,167.42,7.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15863,34759,New York Excelsior,Nenne,Blizzard World,Ashe,12.0,12160.37,0.00,9.0,20.0,8.0,0.00,14290.73,3.0,92.49,6.75
15864,34759,New York Excelsior,Nenne,Hanamura,Ashe,0.0,3742.69,0.00,2.0,1.0,1.0,0.00,2417.33,1.0,4.96,1.50
15865,34759,New York Excelsior,Nenne,Hanamura,Tracer,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.01
15866,34759,New York Excelsior,Nenne,Ilios,Tracer,2.0,1949.50,0.00,5.0,7.0,5.0,0.00,4284.34,2.0,24.35,4.39


In [74]:
# Ignore map and average stats
average_table = data.drop('map_name', axis=1, inplace=False)
average_table = average_table.groupby(['esports_match_id', 'team_name', 'player_name', 'hero_name']).mean()
average_table = average_table.reset_index()
average_table

healers = ['Lúcio', 'Baptiste', 'Moira', 'Ana', 'Zenyatta', 'Mercy', 'Brigitte', 'Soldier: 76']
tanks = ['Reinhardt', 'Mei', 'Orisa', 'Symmetra', 'Winston', 'D.Va', 'Zarya', 'Brigitte', 'Sigma']

# Compare efficiencies of players playing as healers
healers_table = average_table.loc[average_table['hero_name'].isin(healers)].copy()

def healer_efficiency(row):
    return (2*row['Assists'] + 1.5*row['Barrier Damage Done'] - 2*row['Deaths'] + 2*row['Eliminations']
            + row['Final Blows'] + 2.5*row['Healing Done'] + 1.5*row['Objective Kills'] + row['Objective Time'] + 1.5*row['Ultimates Earned'])

healers_table['Efficiency'] = healers_table.apply(lambda row: healer_efficiency(row), axis=1)

# Ignore rows with less than 1500 efficiency as that probably means the player only played as the healer for less than a minute
# and drop newly irrelevant columns
healers_table = healers_table[healers_table.Efficiency > 2000]
healers_table = healers_table[['team_name', 'player_name', 'Efficiency']]
healers_table = healers_table.groupby(['team_name', 'player_name']).mean()
healers_table = healers_table.reset_index()

plot_table = healers_table[['team_name', 'Efficiency']]
plot_table = plot_table.groupby(['team_name']).agg(list)
plot_table

stat_name,team_name,Efficiency
0,Atlanta Reign,26220.102083
1,Atlanta Reign,22993.188000
2,Atlanta Reign,27484.393667
3,Atlanta Reign,22546.559623
4,Boston Uprising,17109.048111
...,...,...
91,Vancouver Titans,19405.694167
92,Vancouver Titans,4356.690000
93,Washington Justice,22686.944017
94,Washington Justice,18306.466047


In [5]:
# FIND MOST IMPACTFUL PLAYER
teams = data.loc[:, 'team_name'].unique()
maps = data.loc[:, 'map_name'].unique()
maps

array(['Eichenwalde', 'Havana', 'Horizon Lunar Colony', 'Lijiang Tower',
       'Junkertown', "King's Row", 'Dorado', 'Hanamura', 'Nepal',
       'Blizzard World', 'Oasis', 'Temple of Anubis', 'Busan', 'Ilios',
       'Numbani', 'Paris', 'Route 66', 'Volskaya Industries', 'Hollywood',
       'Rialto', 'Watchpoint: Gibraltar'], dtype=object)